### Установка

In [ ]:
!pip install setfit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Импорты и агрегация данных

In [ ]:
from sentence_transformers.losses import CosineSimilarityLoss
from sklearn.model_selection import train_test_split
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset
import pandas as pd
import numpy as np
import joblib
import json
import pickle
import gc

In [ ]:
with open('/content/contract_enforcement.pkl', 'rb') as fp:
  contract_enforcement = pickle.load(fp)
print(len(contract_enforcement))
contract_enforcement = pd.Series(contract_enforcement)

contract_enforcement = contract_enforcement[~contract_enforcement.isna()]
contract_enforcement = contract_enforcement.values
len(contract_enforcement)

988


984

In [ ]:
with open('/content/garantee_enforcement.pkl', 'rb') as fp:
  garantee_enforcement = pickle.load(fp)
print(len(garantee_enforcement))
garantee_enforcement = pd.Series(garantee_enforcement)

garantee_enforcement = garantee_enforcement[~garantee_enforcement.isna()]
garantee_enforcement = garantee_enforcement.values
len(garantee_enforcement)

811


508

In [ ]:
with open('/content/nonetype.pkl', 'rb') as fp:
  nonetype_sentences = pickle.load(fp)
nonetype = []

for i in range(len(nonetype_sentences)):
  for j in range(len(nonetype_sentences[i])):
    if len(nonetype_sentences[i][j]) > 5:
      nonetype.append(nonetype_sentences[i][j])

nonetype = np.array(nonetype)
len(nonetype)

11232

In [ ]:
nonetype[:5]

array(['Версия с 04.07.2022 года',
       'Ю «Государственное автономное учреждение',
       "Саха (Якутия) ''Республиканская больница №1 -",
       "Национальный центр медицины''»",
       'Подписано усиленной квалифицированной электронной'], dtype='<U867')

In [ ]:
nonetype_1000 = np.random.choice(nonetype, 1000)
df_for_contract_enforcement = pd.DataFrame(list(set(contract_enforcement)), columns=['sentence'])
df_for_nonetype = pd.DataFrame(nonetype_1000, columns=['sentence'])

In [ ]:
df_for_contract_enforcement['label'] = 1
df_for_nonetype['label'] = 0

In [ ]:
df_for_contract_enforcement.head()

,sentence,label
0,Размер обеспечения исполнения Контракта состав...,1
1,Размер обеспечения договора 5 % от НМЦ составл...,1
2,Если в извещении о закупке установлено требова...,1
3,Поставщик при заключении Договора должен предо...,1
4,Размер обеспечения исполнения договора составл...,1


In [ ]:
df_for_nonetype.head()

,sentence,label
0,ТЕХНОЛОГИЙ» «УТВЕРЖДАЮ»,0
1,"Доверенности от 25.04.2022г. № 8/од, с одной С...",0
2,действующего (,0
3,"МРБ» адрес: 662161, г.",0
4,включенной в сведении о закупке функционала,0


In [ ]:
df_contract_enforcement_model = pd.concat([df_for_contract_enforcement, df_for_nonetype], axis=0)

In [ ]:
df_contract_enforcement_model = df_contract_enforcement_model.sample(frac=1).reset_index(drop=True)

In [ ]:
nonetype_500 = np.random.choice(nonetype, 500)
df_for_garantee_enforcement = pd.DataFrame(list(set(garantee_enforcement)), columns=['sentence'])
df_for_nonetype = pd.DataFrame(nonetype_500, columns=['sentence'])

df_for_garantee_enforcement['label'] = 1
df_for_nonetype['label'] = 0

df_for_garantee_enforcement_model = pd.concat([df_for_garantee_enforcement, df_for_nonetype], axis=0)



In [ ]:
df_for_garantee_enforcement_model = df_for_garantee_enforcement_model.sample(frac=1).reset_index(drop=True)


### SetFit model

In [ ]:
train, test = train_test_split(df_contract_enforcement_model, test_size=0.98)

In [ ]:
len(train), len(test)

(31, 1567)

In [ ]:
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"sentence": "text", "label": "label"},
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
trainer.train()
metrics = trainer.evaluate()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 1240
  Num epochs = 1
  Total optimization steps = 78
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Applying column mapping to evaluation dataset
***** Running evaluation *****


Так как модель SetFit отлично работает на малом количестве данных, можем оставить основную часть датасета в валидационной выборке

In [ ]:
metrics

{'accuracy': 0.982769623484365}

In [ ]:
del trainer

In [ ]:
gc.collect()


217

модель показывает accuracy 0.98 на наборе из 31 строки данных. 


### Batch size = 4
попробуем уменьшить размер батча, чтобы не перегружать ОЗУ

In [ ]:
train, test = train_test_split(df_contract_enforcement_model, test_size=0.9)

In [ ]:
len(train), len(test)

(159, 1439)

In [ ]:
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    batch_size=4,
    column_mapping={"sentence": "text", "label": "label"},
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
trainer.train()
metrics = trainer.evaluate()

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 6360
  Num epochs = 1
  Total optimization steps = 1590
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1590 [00:00<?, ?it/s]

Applying column mapping to evaluation dataset
***** Running evaluation *****


In [ ]:
metrics

{'accuracy': 0.9986101459346769}

In [ ]:
joblib.dump(trainer, 'model.joblib')

['model.joblib']

In [ ]:
from google.colab import files
files.download('model.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Загрузка contract_enforcement модели из дампа

In [ ]:
contract_enforcement_trainer = joblib.load('./contract_enforcement_setfit_model.joblib')

In [ ]:
contract_enforcement_trainer